In [17]:
import pandas as pd
import numpy as np

In [18]:



def getPricesFromCSV(file_path, reference_dates):
    # get all the data from the csv file where the dates are in the reference_dates list
    df = pd.read_csv(reference_dates, skiprows=15)
    df["Date"] = pd.to_datetime(df["Date"])
    date_series = df["Date"]

    df = pd.read_csv(file_path)
    # if format of date is MM/DD/YY, convert it to datetime
    if type(df["Date"].iloc[0]) == numpy.int64:

        df["Date"] = pd.to_datetime(df["Date"], format="%Y%m%d")
    # if format of date is YYYYMMDD, convert it to datetime
    else:

        df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%y")

    df_filtered = df[df["Date"].isin(date_series)]
    return df_filtered

In [19]:
dataFrame_H5A4 = getPricesFromCSV("C:/Users/nimas/Regentfund Repository/fixed-income/Data Files/H5A4.csv", "/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")
dataFrame_SPY = getPricesFromCSV("/fixed-income/Data Files/SPY 1.csv", "/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")
dataFrame_G5O2 = getPricesFromCSV("/fixed-income/Data Files/G5O2.csv", "/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")
dataFrame_TLT = getPricesFromCSV("/fixed-income/Data Files/TLT_simplified.csv", "/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")
dataFrame_LQD = getPricesFromCSV("/fixed-income/Data Files/LQD_simplified.csv", "/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")
dataFrame_C5A0 = getPricesFromCSV("/fixed-income/Data Files/C5A0.csv", "/Volumes/Raghav_DATA/Capitalfund/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls")

FileNotFoundError: [Errno 2] No such file or directory: '/fixed-income/Data Files/MLH5A4_Price_History(Fixed Income).xls'

In [3]:
def create_bmom(dataframe, weights, price_col="Price"):
    data = dataframe.dropna().sort_values("Date").set_index("Date")

    px = data[price_col]
    momentums = {}
    weighted_momentums = {}

    for lookback, weight in weights.items():
        momentums[lookback] = px.pct_change(lookback)
        weighted_momentums[lookback] = momentums[lookback] * weight

    wm_df = pd.concat(weighted_momentums, axis=1)

    # Find first index where ALL lookbacks are valid
    first_valid = wm_df.dropna().index[0]

    # Sum only from that index onward
    bmom = wm_df.loc[first_valid:].sum(axis=1)

    daily_returns = px.pct_change(1)
    dates = bmom.index

    return bmom, momentums, dates, daily_returns

bmom_H5A4, momentums_H5A4, dates_H5A4, daily_returns_H5A4 = create_bmom(
    dataFrame_H5A4,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "H5A4"
)

bmom_SPY, momentums_SPY, dates_SPY, daily_returns_SPY = create_bmom(
    dataFrame_SPY,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "SPY_adj_close"
)

bmom_G5O2, momentums_G5O2, dates_G5O2, daily_returns_G5O2 = create_bmom(
    dataFrame_G5O2,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "G5O2"
)


bmom_TLT, momentums_TLT, dates_TLT, daily_returns_TLT = create_bmom(
    dataFrame_TLT,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "TLT"
)
bmom_LQD, momentums_LQD, dates_LQD, daily_returns_LQD = create_bmom(
    dataFrame_LQD,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "LQD"
)
bmom_C5A0, momentums_C5A0, dates_C5A0, daily_returns_C5A0 = create_bmom(
    dataFrame_C5A0,
    {20: 0.15, 60: 0.35, 120: 0.35, 240: 0.15},
    "C5A0"
)





NameError: name 'dataFrame_H5A4' is not defined

In [1]:
def strat1_signals(bmom_asset, bmom_benchmark, mom240_asset, thresh=-0.03, init_wgt=0):
    # Use asset index as the master timeline (like the sheet's Date column)
    idx = bmom_asset.index
    a = bmom_asset.astype(float)
    s = bmom_benchmark.reindex(idx).astype(float)
    m240 = mom240_asset.reindex(idx).astype(float)

    # Cross-down conditions (NaNs will naturally evaluate to False in comparisons)
    cond1 = (s.shift(1) > thresh) & (s < thresh) & (a < thresh)
    cond2 = (a.shift(1) > thresh) & (a < thresh) & (s < thresh)

    risk_off = (cond1 | cond2) & (m240 < 0)

    # Desired signal: 0 on risk_off, 1 if a>0, else NaN (to be forward-filled)
    sig = pd.Series(np.nan, index=idx, dtype="float64")
    sig = sig.mask(risk_off, 0)
    sig = sig.mask(~risk_off & (a > 0), 1)

    # Spreadsheet-style initialization + carry forward
    sig.iloc[0] = init_wgt
    sig = sig.ffill().astype(int)

    #make sure to align with smaller of the two date ranges between asset and benchmark
    min_idx = idx.intersection(bmom_benchmark.index)
    sig = sig.reindex(min_idx)

    return sig


In [2]:
signals_strat1_H5A4 = strat1_signals(bmom_H5A4, bmom_SPY, momentums_H5A4[240], thresh=-0.03)
signals_strat1_G5O2 = strat1_signals(bmom_G5O2, bmom_TLT, momentums_G5O2[240], thresh=-0.03)
signals_strat1_C5A0 = strat1_signals(bmom_C5A0, bmom_LQD, momentums_C5A0[240], thresh=-0.03)


NameError: name 'bmom_H5A4' is not defined

In [51]:
def strat2_from_strat1(strat1: pd.Series,
                       hyg_ret: pd.Series,
                       drawdiffthresh: float = 0.10) -> pd.Series:
    idx = strat1.index
    r_hyg = hyg_ret.reindex(idx).fillna(0.0)

    # Strat1 returns use previous-day weight (typical backtest convention)
    r_strat1 = r_hyg * strat1.shift(1).fillna(0)

    def drawdown(r: pd.Series) -> pd.Series:
        eq = (1.0 + r).cumprod()
        return eq / eq.cummax() - 1

    dd_strat1 = drawdown(r_strat1)
    dd_hyg = drawdown(r_hyg)


    dd_diff = dd_strat1 - dd_hyg


    override_trigger = dd_diff > drawdiffthresh
    
    # Identify start of each override regime
    override_start = override_trigger & (~override_trigger.shift(1, fill_value=False))
    
    # Each override regime gets a unique ID
    regime_id = override_start.cumsum()
    
    # Determine where an override regime should end:
    # first date AFTER regime start where strat1 == 1
    end_override = (regime_id > 0) & (strat1 == 1)
    
    # For each regime, find first exit index
    first_exit = (
        end_override.groupby(regime_id)
        .transform(lambda x: x & (~x.shift(1, fill_value=False)))
    )
    
    # Build active override mask
    active_override = (
        (regime_id > 0)
        & ~(first_exit.groupby(regime_id).cumsum().astype(bool))
    )
    
    # Strat2 logic
    strat2 = strat1.copy()
    strat2[active_override] = 1
    strat2[override_trigger] = 1   # threshold always wins
    
    strat2.iloc[0] = 0
    
    return strat2



strat2_signals_H5A4 = (strat2_from_strat1(signals_strat1_H5A4, daily_returns_H5A4, drawdiffthresh=0.10))
strat2_signals_G5O2 = (strat2_from_strat1(signals_strat1_G5O2, daily_returns_G5O2, drawdiffthresh=0.10))
strat2_signals_C5A0 = (strat2_from_strat1(signals_strat1_C5A0, daily_returns_C5A0, drawdiffthresh=0.10))


In [ ]:
def analyze_strat(signals: pd.Series, daily_returns: pd.Series, file_path):
    idx = signals.index
    r_strat = daily_returns.reindex(idx).fillna(0.0) * signals.shift(1).fillna(0)
    
    def drawdown(r: pd.Series) -> pd.Series:
        eq = (1.0 + r).cumprod()
        return eq / eq.cummax() - 1

    def equity_curve(r: pd.Series) -> pd.Series:
        return (1.0 + r).cumprod()
    
    def annualized_return(r: pd.Series) -> float:
        equity = equity_curve(r)
        return equity.iloc[-1] ** (252 / len(r)) - 1  # Annualized return
    def volatility(r: pd.Series) -> float:
        return r.std() * np.sqrt(252)  # Annualized volatility
    def sharpe_ratio(r: pd.Series, risk_free_rate=0.0) -> float:
        ann_return = annualized_return(r)
        ann_vol = volatility(r)
        return ann_return / ann_vol if ann_vol > 0 else np.nan
    print("file:", file_path)
    print(f"Annualized Return: {annualized_return(r_strat):.2}")
    print(f"Annualized Volatility: {volatility(r_strat):.2}")
    print(f"Sharpe Ratio: {sharpe_ratio(r_strat):.2}")
    
    #Write to CSV
    output_df = pd.DataFrame({
        "Date": idx,
        "Signal": signals,
        "Daily Return": r_strat,
        "Drawdown": drawdown(r_strat),
        "Equity Curve": equity_curve(r_strat)
    })
    output_df.to_csv(file_path, index=False)


analyze_strat(strat2_signals_H5A4, daily_returns_H5A4, "/Volumes/Raghav_DATA/Capitalfund/fixed-income/strat2_output.csv")
analyze_strat(signals_strat1_H5A4, daily_returns_H5A4, "/Volumes/Raghav_DATA/Capitalfund/fixed-income/strat1_output.csv")


file: /Volumes/Raghav_DATA/Capitalfund/fixed-income/strat2_output.csv
Annualized Return: 0.012
Annualized Volatility: 0.044
Sharpe Ratio: 0.28
file: /Volumes/Raghav_DATA/Capitalfund/fixed-income/strat1_output.csv
Annualized Return: 0.0087
Annualized Volatility: 0.036
Sharpe Ratio: 0.24


In [52]:
print(strat2_signals_G5O2)
print(strat2_signals_C5A0)
print(strat2_signals_H5A4)

Date
2008-03-25    0
2008-03-26    1
2008-03-27    1
2008-03-28    1
2008-03-31    1
             ..
2025-11-05    1
2025-11-06    1
2025-11-07    1
2025-11-10    1
2025-11-11    1
Length: 4439, dtype: int64
Date
2008-03-25    0
2008-03-26    1
2008-03-27    1
2008-03-28    1
2008-03-31    1
             ..
2025-11-05    1
2025-11-06    1
2025-11-07    1
2025-11-10    1
2025-11-11    1
Length: 4439, dtype: int64
Date
1997-12-11    0
1997-12-12    1
1997-12-15    1
1997-12-16    1
1997-12-17    1
             ..
2025-11-05    1
2025-11-06    1
2025-11-07    1
2025-11-10    1
2025-11-11    1
Length: 7023, dtype: int64


In [60]:

import numpy as np
import pandas as pd


def strat_3(bmom, signals, daily_returns):

    assets = ["H5A4", "C5A0", "G5O2"]

    # Use index from one of the series
    idx = bmom[assets[0]].index

    # -----------------------------
    # 1. Linear momentum scores
    # -----------------------------
    scores = pd.DataFrame(index=idx, columns=assets, dtype=float)

    for asset in assets:
        excess_bmom = np.maximum(0.0, bmom[asset].reindex(idx))
        scores[asset] = excess_bmom * signals[asset].reindex(idx)

    # -----------------------------
    # 2. Weight calculation
    # -----------------------------
    weights = pd.DataFrame(index=idx,
                           columns=["Cash"] + assets,
                           dtype=float)

    for i in range(len(scores)):
        row_scores = scores.iloc[i]
        total = row_scores.sum()

        # Defensive mode
        if total == 0:
            weights.iloc[i] = [0.10, 0.0, 0.0, 0.90]
            continue

        # Active mode
        cash = 0.05
        raw = (row_scores / total) * 0.95

        active_assets = raw[raw > 0].index.tolist()

        # Enforce 5% minimum
        for a in active_assets:
            if raw[a] < 0.05:
                deficit = 0.05 - raw[a]
                raw[a] = 0.05

                others = [x for x in active_assets if x != a]
                if len(others) > 0:
                    other_total = raw[others].sum()
                    for o in others:
                        raw[o] -= deficit * (raw[o] / other_total)

        weights.iloc[i] = [cash,
                           raw["H5A4"],
                           raw["C5A0"],
                           raw["G5O2"]]
        
    # -----------------------------
    # 3. Portfolio returns
    # -----------------------------
    portfolio_return = (
        weights["H5A4"].shift(1) * daily_returns["H5A4"].reindex(idx) +
        weights["C5A0"].shift(1) * daily_returns["C5A0"].reindex(idx) +
        weights["G5O2"].shift(1) * daily_returns["G5O2"].reindex(idx)
    ).fillna(0.0)

    equity_curve = (1 + portfolio_return).cumprod()
    weights = weights.fillna(0.0)

    return weights, portfolio_return, equity_curve, scores
weights3, ret3, eq3, scores3 = strat_3(
    bmom={
        "H5A4": bmom_H5A4,
        "C5A0": bmom_C5A0,
        "G5O2": bmom_G5O2
    },
    signals={
        "H5A4": strat2_signals_H5A4,
        "C5A0": strat2_signals_C5A0,
        "G5O2": strat2_signals_G5O2
    },
    daily_returns={
        "H5A4": daily_returns_H5A4,
        "C5A0": daily_returns_C5A0,
        "G5O2": daily_returns_G5O2
    }
)

print(weights3)


            Cash      H5A4      C5A0      G5O2
Date                                          
1997-12-11  0.10  0.000000  0.000000  0.900000
1997-12-12  0.05  0.950000  0.000000  0.000000
1997-12-15  0.05  0.950000  0.000000  0.000000
1997-12-16  0.05  0.950000  0.000000  0.000000
1997-12-17  0.05  0.950000  0.000000  0.000000
...          ...       ...       ...       ...
2025-11-05  0.05  0.267542  0.361847  0.320610
2025-11-06  0.05  0.224967  0.375060  0.349973
2025-11-07  0.05  0.232338  0.362998  0.354664
2025-11-10  0.05  0.280022  0.341943  0.328036
2025-11-11  0.05  0.285006  0.342490  0.322504

[7023 rows x 4 columns]
